In [1]:
import pandas as pd
import os
import sys
sys.path.append('../../src')
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL
from data_process import data_split, neg_sample

## Loading the application 

In [10]:
apps = pd.read_csv('../../data/jobs/tsv/apps.tsv', sep='\t', encoding='utf-8')
apps.rename(columns = {'UserID':DEFAULT_USER_COL, 'ApplicationDate':DEFAULT_TIMESTAMP_COL, \
                      'JobID':DEFAULT_ITEM_COL
                      }, inplace=True)

In [11]:
apps.columns

Index(['userid', 'WindowID', 'Split', 'timestamp', 'itemid'], dtype='object')

## Loading the jobs info

all columns: ['JobID', 'WindowID', 'Title', 'Description', 'Requirements','City', 'State', 'Country', 'Zip5', 'StartDate','EndDate']

selected_columns = ['JobID', 'WindowID','City_job', 'State_job', 'Country_job', 'Zip5', 'StartDate','EndDate']

In [12]:
selected_feature = ['JobID', 'WindowID','City', 'State', 'Country', 'Zip5', 'StartDate','EndDate']
jobs = pd.read_csv('../../data/jobs/tsv/jobs.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip', usecols = selected_feature)

jobs.rename(columns = {'JobID':DEFAULT_ITEM_COL, 'City':'City_job', \
                       'State':'State_job', 'Country':'Country_job', \
                      'Zip5':'Zip_job'}, inplace=True)

/tmp/ipykernel_52291/430080897.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv('../../data/jobs/tsv/jobs.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip', usecols = selected_feature)


In [13]:
jobs.columns

Index(['itemid', 'WindowID', 'City_job', 'State_job', 'Country_job', 'Zip_job',
       'StartDate', 'EndDate'],
      dtype='object')

## Loading the user history info

full columns: Index(['UserID', 'WindowID', 'Split', 'Sequence', 'JobTitle'], dtype='object')

selected columns:['UserID', 'WindowID', 'Split', 'Sequence']

In [14]:
selected_col=['UserID', 'WindowID', 'Split', 'Sequence']
user_history = pd.read_csv('../../data/jobs/tsv/user_history.tsv', sep='\t', encoding='utf-8', usecols=selected_col)


### With jobtitle

In [15]:
# user_history.rename(columns = {'UserID':DEFAULT_USER_COL, 'JobTitle':'JobTitle_user'}, inplace=True)
# user_history = user_history[user_history['JobTitle_user'].notna()]
# user_history_title = user_history.groupby('userid')['JobTitle_user'].\
# apply(lambda x:','.join(x)).reset_index()


## Loading the user profile info

In [16]:
users = pd.read_csv('../../data/jobs/tsv/users.tsv', sep='\t', encoding='utf-8')
users.rename(columns = {'UserID':DEFAULT_USER_COL, 'ZipCode':'Zip_user'}, inplace=True)

In [17]:
users.columns

Index(['userid', 'WindowID', 'Split', 'City', 'State', 'Country', 'Zip_user',
       'DegreeType', 'Major', 'GraduationDate', 'WorkHistoryCount',
       'TotalYearsExperience', 'CurrentlyEmployed', 'ManagedOthers',
       'ManagedHowMany'],
      dtype='object')

## Merge data 

### Have job title

In [18]:
# df_merge = apps.merge(users, how='left', on=[DEFAULT_USER_COL]) \
# .merge(user_history_title, how='left', on=[DEFAULT_USER_COL]) \
# .merge(jobs, how='left', on=[DEFAULT_ITEM_COL])

### Don't have job title

In [19]:
df_merge = apps.merge(users, how='left', on=[DEFAULT_USER_COL]) \
.merge(jobs, how='left', on=[DEFAULT_ITEM_COL])

In [20]:
df_merge = df_merge.drop(['WindowID_x', 'Split_x'], axis=1).rename(columns = {'WindowID_y':'WindowID_user', \
                                                            'Split_y':'Split', 'Title':'JobTitle', \
                                                            'WindowID':'WindowID_job'
                                                            })

In [21]:
df_merge.columns

Index(['userid', 'timestamp', 'itemid', 'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany', 'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate'],
      dtype='object')

## Save data

In [22]:
df_merge = df_merge[df_merge['Country']=='US']
df_merge[DEFAULT_RATING_COL] = '1'

In [24]:
df_merge.to_csv("../../data/jobs/merged_all.csv", index=False)

In [25]:
# aa.to_hdf("../data/merged_apps.h5", key='all', format='table', data_columns=True, mode='w')


## Convert the datatype 

In [26]:
# cat_features = ['userid', 'itemid', 'WindowID_user', 'Split', 'City', \
#                 'State', 'Country', 'Zip_user', 'DegreeType', 'Major', \
#                'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed', \
#                'ManagedOthers', 'ManagedHowMany','WindowID_job','City_job', \
#                'State_job', 'Country_job', 'Zip_job']
# # num_features = []
# time_features = ['timestamp', 'GraduationDate','StartDate', 'EndDate' ]
# # str_features = ['JobTitle_user', 'JobTitle','Description','Requirements']

In [27]:
# all_features = [cat_features, time_features, str_features]

In [28]:
# # for feature_type in all_features:
# for f in cat_features:
# #     aa[f] = aa[f].fillna('none').astype('category')
# #     pd.Categorical(aa[f])
#     df_merge[f] = df_merge[f].astype('str')
# for f in time_features:
#     df_merge[f] = pd.to_datetime(df_merge[f])
# # for f in str_features:
# #     aa[f] = aa[f].astype(str)

## Subsample dataset 
Keep only US dataset and subsmaple 3-window users 
all the columns: Index(['userid', 'timestamp', 'itemid', 'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany', 'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate'],
      dtype='object')

In [42]:
# df_all = pd.read_hdf('../data/merged_apps.h5', key='all', columns=['userid', 'itemid', \
#                                                                    'WindowID_user','Country'])

In [29]:
# df_all = pd.read_hdf('../data/merged_apps.h5', key='all')
df_all = pd.read_csv("../../data/jobs/merged_all.csv", low_memory=False)

In [30]:
df_all.columns

Index(['userid', 'timestamp', 'itemid', 'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany', 'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',
       'rating'],
      dtype='object')

Subsample certain windows dataset 

In [31]:
select_win = [3, 4, 5]

In [32]:
df_win = df_all[df_all.WindowID_user.isin(select_win)]

In [33]:
df_win.to_csv("../../data/jobs/merged_sub.csv", index=False)

In [62]:
# df_win.isnull().sum()

## Crearte the leave one out data

In [ ]:
path_read = "../../data/jobs/merged_sub.csv"
path_save1 = "../../data/jobs/leave_one_train.csv"
path_save2 = "../../data/jobs/leave_one_test.csv"
data_split.main(path_read, path_save1, path_save2)

# Create the negative sample dataset 

### Create for the training set 

In [17]:
# path_read_test = '../../Data/jobs/leave_one_test.csv'
path_read_all = '../../data/jobs/merged_sub.csv'
path_read_train = '../../data/jobs/leave_one_train.csv'
path_save = '../../data/jobs/leave_one_train_neg.csv'
ng_num = 4
neg_sample.main(path_read_all = path_read_all,
                path_read_goal = path_read_train, 
                path_save=path_save, ng_num=ng_num, 
                path_read_neg=None, test=False, )

100%|█████████████████████████████████████| 89946/89946 [18:18<00:00, 81.86it/s]


### Create for the test set 

In [18]:
path_read_test = '../../data/jobs/leave_one_test.csv'
path_read_all = '../../data/jobs/merged_sub.csv'
path_save = '../../data/jobs/test_pos_neg.csv'
train_neg = '../../data/jobs/leave_one_train_neg.csv'
ng_num = 100
neg_sample.main(path_read_all=path_read_all, 
                path_read_goal=path_read_test, 
                path_save=path_save, 
                ng_num=ng_num, 
                test=True,
                path_read_neg = train_neg)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 8: invalid start byte

In [19]:
pd.read_csv('../../data/jobs/leave_one_train_neg.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 8: invalid start byte

# Processing the missing value 